In [42]:
!pip install scikit-multilearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [43]:
import pandas as pd
from google.colab import drive
from sklearn.naive_bayes import ComplementNB
from sklearn.preprocessing import normalize,OneHotEncoder,MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split,RandomizedSearchCV
import gensim
from skmultilearn.problem_transform import LabelPowerset
from sklearn.metrics import accuracy_score,f1_score
from sklearn.naive_bayes import ComplementNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC,SVC
from scipy.sparse import hstack,csr_matrix
import numpy as np
from scipy.stats import reciprocal, uniform
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [44]:
#Import dataset
original = pd.read_json('/content/drive/MyDrive/sml-authorship-attribution/Data/train.json')
prediction = pd.read_json('/content/drive/MyDrive/sml-authorship-attribution/Data/test.json')

In [45]:
#Preprocessing the columns
original['id']=original.index
original['coauthors']=original['authors']
original['authors']=original['authors'].apply(lambda x: [i for i in x if i<100] if len([i for i in x if i<100])!=0 else [-1])
original["authors_text"]=original['authors'].apply(lambda x: ','.join([str(i) for i in x]))
original['coauthors']=original.apply(lambda x: list(set(x.coauthors)-set(x.authors)),axis=1)
original['venue'] = original['venue'].astype(str)

In [46]:
original

,authors,year,abstract,venue,title,id,coauthors,authors_text
0,"[42, 36]",9,"[2455, 1858, 2335, 1543, 1800, 1860, 2000, 286...",20,"[41, 1550, 1563, 1594, 1544, 1919, 1644, 37, 1...",0,[13720],"42,36"
1,[45],15,"[40, 1542, 1691, 2449, 1535, 3616, 2206, 1904,...",2,"[1731, 47, 11, 57, 4624, 1525, 1535, 47, 11, 3...",1,"[15881, 1359]",45
2,[-1],17,"[40, 1542, 1691, 2449, 1535, 2610, 1543, 1535,...",,"[2085, 1719, 1846, 1745, 2243, 1553, 1606, 159...",2,"[17763, 19166]",-1
3,[97],10,"[46, 1624, 1547, 56, 1687, 1644, 6, 7, 3386, 1...",4,"[40, 1733, 1735, 1540, 1655, 46, 1624, 1547, 5...",3,[],97
4,[2],10,"[37, 3709, 3836, 1586, 2151, 1727, 3021, 1860,...",9,"[38, 1592, 2088, 1543, 1574, 1727, 1597, 1813,...",4,[19617],2
...,...,...,...,...,...,...,...,...
25788,[78],14,"[46, 1605, 1681, 10, 1557, 4741, 1535, 2021, 1...",1,"[46, 1910, 36, 2107, 1547, 1553, 1716, 1528, 5...",25788,[1797],78
25789,[-1],11,"[46, 1605, 1691, 10, 2551, 4168, 2030, 3134, 1...",278,"[47, 1574, 1541, 1549, 47, 1966, 2114, 112, 12...",25789,"[7905, 11563, 18357]",-1
25790,[-1],7,"[1611, 1543, 1535, 2817, 2073, 1647, 11, 2933,...",17,"[56, 1718, 3012, 56, 1902, 2160, 1546, 1623, 1...",25790,"[2236, 20436, 10501]",-1
25791,[-1],16,"[37, 1662, 33, 2007, 1669, 4981, 1650, 1527, 1...",0,"[3066, 1728, 11, 1546, 11, 3066, 1728, 1531, 1...",25791,"[8760, 3921]",-1


In [47]:
#Sampling rows with no prolific authors i.e, tagged -1 to introduce into the train dataset. 
#We dont use the entire set to avoid class imbalance
negative_samples=original[original['authors_text']=='-1'].sample(frac=0.1)
print(negative_samples.shape)
original=original[original['authors_text']!='-1']
print(original.shape)
original=pd.concat([original,negative_samples])
print(original.shape)

(1833, 8)
(7460, 8)
(9293, 8)


In [48]:
original = original.reset_index(drop = True)
original = original.drop(['id'], axis = 1)

In [49]:
# original['abstract'] = original['abstract'].apply(lambda x: [str(i) for i in x])
# original['title'] = original['title'].apply(lambda x: [str(i) for i in x])

# #prediction

# prediction['abstract'] = prediction['abstract'].apply(lambda x: [str(i) for i in x])
# prediction['title'] = prediction['title'].apply(lambda x: [str(i) for i in x])

In [50]:
# corpus_abstract = original["abstract"].values
# w2v_abstract = gensim.models.word2vec.Word2Vec(corpus_abstract, size=300, window=8, min_count=1, workers=20, iter=10)
# corpus_title = original["title"].values
# w2v_title = gensim.models.word2vec.Word2Vec(corpus_title, size=80, window=8, min_count=1, workers=20, iter=10)

In [51]:
original['abstract'] = original['abstract'].apply(lambda x: ' '.join([str(i) for i in x]))
original['title'] = original['title'].apply(lambda x: ' '.join([str(i) for i in x]))

#prediction
prediction['abstract'] = prediction['abstract'].apply(lambda x: ' '.join([str(i) for i in x]))
prediction['title'] = prediction['title'].apply(lambda x: ' '.join([str(i) for i in x]))

In [52]:
#Function to convert a use word vectors to geberate a sentence vector. 
#We did use doc2vec for the same exercise but it did prove an improvement in the results
def sentence2Vec(w2vModel,abstract):    
    trans_abstract = np.zeros(w2vModel.wv.vectors.shape[1])
    # Looping thru each word in the sentence and if its present in 
    # the Word2Vec model then storing its vector
    count=0
    for word in abstract:
        if word in w2vModel.wv.vocab.keys():    
            trans_abstract=trans_abstract+w2vModel.wv[word]
            count+=1
    # Averaging Word vectors
    trans_abstract/=count
    return (trans_abstract)

In [53]:
#train test split
train, test = train_test_split(original, random_state=42, test_size=0.2, shuffle=True)

print(train.shape)
print(test.shape)

(7434, 7)
(1859, 7)


In [54]:
coauthors=set()
for i in train['coauthors'].values:
  for x in i:
    coauthors.add(x)
years=train['year'].unique()
for i in years:
  train['year_'+str(i)]=train['year'].apply(lambda x: 1 if x==i else 0)
  test['year_'+str(i)]=test['year'].apply(lambda x: 1 if x==i else 0)
  prediction['year_'+str(i)]=prediction['year'].apply(lambda x: 1 if x==i else 0)
venues=train['venue'].unique()
for i in venues:
  train['venue_'+str(i)]=train['venue'].apply(lambda x: 1 if x==i else 0)
  test['venue_'+str(i)]=test['venue'].apply(lambda x: 1 if x==i else 0)
  prediction['venue_'+str(i)]=prediction['venue'].apply(lambda x: 1 if x==i else 0)
for i in coauthors:
  train['coauthors_'+str(i)]=train['coauthors'].apply(lambda x: 1 if i in x else 0)
  test['coauthors_'+str(i)]=test['coauthors'].apply(lambda x: 1 if i in x else 0)
  prediction['coauthors_'+str(i)]=prediction['coauthors'].apply(lambda x: 1 if i in x else 0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1)

In [55]:
year_columns = [col for col in train if col.startswith('year_')]
year_train_le = train[year_columns]
year_test_le = test[year_columns]
year_pred_le = prediction[year_columns]
print(year_train_le.shape, year_test_le.shape)

venue_columns = [col for col in train if col.startswith('venue_')]
venue_train_le = train[venue_columns]
venue_test_le = test[venue_columns]
venue_pred_le = prediction[venue_columns]
print(venue_train_le.shape, venue_test_le.shape)


coauthors_columns = [col for col in train if col.startswith('coauthors_')]
coauthors_train_le = train[coauthors_columns]
coauthors_test_le = test[coauthors_columns]
coauthors_pred_le = prediction[coauthors_columns]
print(coauthors_train_le.shape, coauthors_test_le.shape)

#Multilabel Binarizer to transfrom our label set to a binary matrix
mlb = MultiLabelBinarizer()
y_train = mlb.fit_transform(train['authors'])
y_test = mlb.transform(test['authors'])
print(y_train.shape, y_test.shape )

(7434, 19) (1859, 19)
(7434, 339) (1859, 339)
(7434, 7916) (1859, 7916)
(7434, 101) (1859, 101)


In [56]:
vectorizer1 = TfidfVectorizer()
vectorizer1.fit(train['abstract'])
abstract_train_enc = vectorizer1.transform(train['abstract'])
abstract_test_enc= vectorizer1.transform(test['abstract'])
abstract_pred_enc= vectorizer1.transform(prediction['abstract'])
print(abstract_train_enc.shape)
print(abstract_test_enc.shape)

(7434, 3909)
(1859, 3909)


In [57]:
vectorizer2 = TfidfVectorizer()
vectorizer2.fit(train['title'])
title_train_enc = vectorizer2.transform(train['title'])
title_test_enc = vectorizer2.transform(test['title'])
title_pred_enc = vectorizer2.transform(prediction['title'])
print(title_train_enc.shape)
print(title_test_enc.shape)

(7434, 2295)
(1859, 2295)


In [58]:
# train['abstract_w2v']=train['abstract'].apply(lambda x: sentence2Vec(w2v_abstract,x))
# test['abstract_w2v']=test['abstract'].apply(lambda x: sentence2Vec(w2v_abstract,x))
# prediction['abstract_w2v']=prediction['abstract'].apply(lambda x: sentence2Vec(w2v_abstract,x))
# train['title_w2v']=train['title'].apply(lambda x: sentence2Vec(w2v_title,x))
# test['title_w2v']=test['title'].apply(lambda x: sentence2Vec(w2v_title,x))
# prediction['title_w2v']=prediction['title'].apply(lambda x: sentence2Vec(w2v_title,x))

In [59]:
# train_abstract_w2v=csr_matrix([i for i in train[['abstract_w2v']].T.values[0]])
# test_abstract_w2v=csr_matrix([i for i in test[['abstract_w2v']].T.values[0]])
# prediction_abstract_w2v=csr_matrix([i for i in prediction[['abstract_w2v']].T.values[0]])
# train_title_w2v=csr_matrix([i for i in train[['title_w2v']].T.values[0]])
# test_title_w2v=csr_matrix([i for i in test[['title_w2v']].T.values[0]])
# prediction_title_w2v=csr_matrix([i for i in prediction[['title_w2v']].T.values[0]])

In [60]:
#Using Hstack to combine our sparse matrices to a single compressed sparse representation
x_tr = hstack((year_train_le,venue_train_le, coauthors_train_le, abstract_train_enc, title_train_enc))
x_te = hstack((year_test_le,venue_test_le, coauthors_test_le,  abstract_test_enc, title_test_enc))
x_pred = hstack((year_pred_le,venue_pred_le, coauthors_pred_le,  abstract_pred_enc, title_pred_enc))

print("Final Data matrix")
print(x_tr.shape, y_train.shape)
print(x_te.shape, y_test.shape)
print(x_pred.shape)

Final Data matrix
(7434, 14478) (7434, 101)
(1859, 14478) (1859, 101)
(800, 14478)


In [61]:
%%time
# initialize label powerset multi-label classifier
classifier = LabelPowerset(LinearSVC())

# train
classifier.fit(x_tr.astype(np.float32), y_train.astype(np.float32))

# predict
predictions = classifier.predict(x_te)
# predictions_prob = classifier.predict_proba(x_te)

# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions))
print("F! score = ", f1_score(y_test, predictions, average='samples'))
print("\n")

Accuracy =  0.6761699838622915
F! score =  0.7238197699736161


CPU times: user 16.8 s, sys: 495 ms, total: 17.3 s
Wall time: 16.5 s


In [62]:
#To generate prediction on the test.json in the format for submission
predictions = classifier.predict(x_pred)
pred_df=pd.DataFrame([{'ID':index,'Predict':' '.join([str(x) for x in list(key)])} for index,key in enumerate(mlb.inverse_transform(predictions))])
pred_df.to_csv('submission.csv',index=False)